In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cpu


In [9]:
data = pd.read_csv('/kaggle/input/housing-prices-dataset/Housing.csv')

print(data.head())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [17]:
data = pd.get_dummies(data, drop_first=True)

X = data.drop(columns=['price']) 
y = data['price']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [19]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [20]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [21]:
model = nn.Linear(X_train.shape[1], 1)
model = model.to('cpu') 

In [22]:
criterion = nn.MSELoss()  # Mean Squared Error for regression tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [23]:
# Training the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/100], Loss: 15661489342756.5723
Epoch [2/100], Loss: 5560536381147.4287
Epoch [3/100], Loss: 2439448217892.5713
Epoch [4/100], Loss: 1505969105188.5715
Epoch [5/100], Loss: 1166267145069.7144
Epoch [6/100], Loss: 1009355730944.0000
Epoch [7/100], Loss: 980712121197.7142
Epoch [8/100], Loss: 970720823588.5714
Epoch [9/100], Loss: 981632825051.4286
Epoch [10/100], Loss: 1139012512036.5715
Epoch [11/100], Loss: 1126783076644.5715
Epoch [12/100], Loss: 981857295213.7142
Epoch [13/100], Loss: 971158653220.5714
Epoch [14/100], Loss: 975032304786.2858
Epoch [15/100], Loss: 1045508565284.5714
Epoch [16/100], Loss: 968422738212.5714
Epoch [17/100], Loss: 969071500726.8572
Epoch [18/100], Loss: 1062451050788.5714
Epoch [19/100], Loss: 967279092882.2858
Epoch [20/100], Loss: 979738365366.8572
Epoch [21/100], Loss: 968402711113.1428
Epoch [22/100], Loss: 999525165348.5714
Epoch [23/100], Loss: 969614134710.8572
Epoch [24/100], Loss: 968188940873.1428
Epoch [25/100], Loss: 971201241088.000

In [24]:
model.eval()
predictions = []

with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        predictions.append(outputs.numpy())

predictions = np.concatenate(predictions, axis=0)

mse = criterion(torch.tensor(predictions), y_test_tensor)
print(f'Mean Squared Error on the test set: {mse.item():.4f}')

Mean Squared Error on the test set: 1773550567424.0000
